In [6]:
from azureml.core import Workspace
from aml_obs.management import provision
import pandas as pd
from datetime import timedelta
from aml_obs.collector import Online_Collector

ws = Workspace.from_config()

### Provision Resources

In [ ]:
provision(ws)

In [7]:
dataset = pd.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv")

In [31]:
dataset["datetime"] =  [pd.to_datetime('now') - timedelta(days=x) for x in range(len(dataset))]

C:\Users\nserafino\Anaconda3\envs\mlopsenv\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


In [32]:
dataset.to_parquet("data/iris.parquet", index=False)

In [33]:

table_name = "irisdata" #new dataset

sample_pd_data = pd.read_parquet("data/iris.parquet").head(10)

online_collector = Online_Collector(table_name,ws=ws)
online_collector.batch_collect(sample_pd_data)

In [39]:
table_name = "iris_data" #new dataset

sample_pd_data = pd.read_parquet("data/iris.parquet")
online_collector = Online_Collector(table_name,ws=ws)
online_collector.stream_collect_df(sample_pd_data)

.create table iris_data (['sepal_length']: real, ['sepal_width']: real, ['petal_length']: real, ['petal_width']: real, ['species']: string, ['datetime']: datetime)


In [27]:
# get cluster uri to use in browser

online_collector.cluster_uri

'https://adx02.westus2.kusto.windows.net'